# Pointwise Convolution (1×1 Convolution) — Teori ve Uygulama

Bu defterde **Pointwise Convolution (1×1 Convolution)** kavramını ayrıntılı şekilde inceleyeceksin:

- Pointwise convolution nedir?
- Hangi problemden dolayı ortaya çıkmıştır?
- Depthwise ile ilişkisi nedir?
- Matematiksel olarak ne yapar (kanal karışımı / channel mixing)?
- PyTorch ile nasıl tanımlar ve kullanırsın?
- Depthwise + Pointwise birleşimi (Depthwise Separable Convolution) nasıl çalışır?

Hedef, 1×1 konvolüsyonu "basit bir conv" olarak değil,
**modern CNN mimarilerinin kritik yapı taşı** olarak anlamaktır.


## 1. Pointwise Convolution Nedir?

**Pointwise Convolution = 1×1 Convolution** demektir.

- Kernel boyutu: **1×1**
- Uzamsal filtreleme yapmaz; asıl görevi **kanallar üzerinde çalışmaktır**.
- Bir piksel konumunda (h, w), tüm kanalları alıp yeni kanallara dönüştürür.

Bu nedenle pointwise conv için şu şekilde düşünebilirsin:

> Her (h, w) piksel konumunda çalışan küçük bir **fully connected layer** gibi.
> Girdi: C_in boyutlu bir vektör, Çıkış: C_out boyutlu bir vektör.


## 2. Neden Pointwise Convolution'a İhtiyaç Var?

Depthwise convolution şunu yapar:

- Her kanalı **kendi 3×3 filtresiyle** işler.
- Çıktı boyutu: `(N, C_in, H, W)` → `(N, C_in, H, W)`
- Kanallar birbiriyle **hiç karışmaz**.

Bu durumda:

- Uzamsal bilgi işlenmiştir (kenarlar, patternler vs. kanal bazlı çıkarılmıştır),
- Fakat kanallar arası etkileşim yoktur → temsil gücü sınırlıdır.

Bu eksikliği gidermek için:

- Depthwise sonrası **kanalları birbirine karıştıran**,
- Kanal sayısını artıran veya azaltan,
- Hafif ve esnek bir yapı gerekir.

Bu yapıyı sağlayan şey **1×1 pointwise convolution**'dır.


## 3. Matematiksel Mantık: Kanal Karışımı (Channel Mixing)

1×1 conv için ağırlık tensörünün boyutu:

- `weight.shape = (C_out, C_in, 1, 1)`

Her piksel konumunda yapılan işlem:

\[
y_{c_{out}}(h, w) = \sum_{c_{in}} x_{c_{in}}(h, w) \cdot W_{c_{out}, c_{in}}
\]

Bu ne anlama geliyor?

- Uzamsal komşuluk (3×3, 5×5 gibi) **hiç yok**, sadece (h, w) konumundaki kanal vektörü dönüştürülüyor.
- Yani pointwise conv:
  - **Uzamsal boyutu korur**: (H, W) değişmez,
  - **Kanal boyutunu değiştirir**: C_in → C_out,
  - Kanallar arasında tam bağlantı sağlar (channel mixing).


In [2]:
import torch
import torch.nn as nn

# Basit pointwise conv örneği
C_in, C_out = 32, 64
H, W = 32, 32

x = torch.randn(1, C_in, H, W)

pointwise = nn.Conv2d(
    in_channels=C_in,
    out_channels=C_out,
    kernel_size=1,
    stride=1,
    padding=0,
    bias=False
)

y = pointwise(x)

print("Girdi şekli :", x.shape)   # (1, 32, 32, 32)
print("Çıktı şekli :", y.shape)   # (1, 64, 32, 32)

print("Parametre sayısı (pointwise):", sum(p.numel() for p in pointwise.parameters()))


Girdi şekli : torch.Size([1, 32, 32, 32])
Çıktı şekli : torch.Size([1, 64, 32, 32])
Parametre sayısı (pointwise): 2048


## 4. Depthwise + Pointwise = Depthwise Separable Convolution

Birçok mobil mimaride (MobileNet, EfficientNet, vs.) şu yapı kullanılır:

1. **Depthwise Conv (3×3)**  
   - Uzamsal filtreleme yapar (her kanal kendi filtresiyle).  
   - Kanal sayısı genelde aynı kalır: `C_in → C_in`.

2. **Pointwise Conv (1×1)**  
   - Kanalları karıştırır (channel mixing).  
   - Kanal sayısını istenen değere dönüştürür: `C_in → C_out`.

Bu iki adımın birleşimine **depthwise separable convolution** denir.

Parametre karşılaştırması (kernel=3 için):

- Standart 3×3 Conv:  
  \( 9 \cdot C_{in} \cdot C_{out} \)

- Depthwise + Pointwise:  
  - Depthwise: \( 9 \cdot C_{in} \)  
  - Pointwise: \( C_{in} \cdot C_{out} \)  
  - Toplam: \( 9 C_{in} + C_{in} C_{out} \)

Özellikle `C_out` büyük olduğunda, bu yapı **standart 3×3 conv'a göre çok daha hafiftir**.


In [3]:
class DepthwiseSeparableConv(nn.Module):
    """
    Depthwise (3x3) + Pointwise (1x1) bloğu.
    Bu yapı MobileNet, EfficientNet gibi mimarilerde sıkça kullanılır.
    """
    def __init__(self, c_in, c_out, kernel_size=3, stride=1, padding=1):
        super().__init__()
        # 1) Depthwise: sadece uzamsal işlem, kanal sayısı sabit
        self.depthwise = nn.Conv2d(
            in_channels=c_in,
            out_channels=c_in,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            groups=c_in,   # depthwise
            bias=False
        )
        # 2) Pointwise: kanal karışımı ve kanal sayısını değiştirme
        self.pointwise = nn.Conv2d(
            in_channels=c_in,
            out_channels=c_out,
            kernel_size=1,
            bias=False
        )

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

# Küçük karşılaştırma
C_in, C_out = 32, 64
x = torch.randn(1, C_in, 32, 32)

std_conv = nn.Conv2d(C_in, C_out, kernel_size=3, padding=1, bias=False)
ds_conv  = DepthwiseSeparableConv(C_in, C_out, kernel_size=3, padding=1)

y_std = std_conv(x)
y_ds  = ds_conv(x)

def count_params(m):
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

print("Standart 3x3 Conv parametre sayısı:", count_params(std_conv))
print("Depthwise+Pointwise parametre sayısı:", count_params(ds_conv))
print("Standart Conv çıktı şekli        :", y_std.shape)
print("Depthwise+Pointwise çıktı şekli  :", y_ds.shape)


Standart 3x3 Conv parametre sayısı: 18432
Depthwise+Pointwise parametre sayısı: 2336
Standart Conv çıktı şekli        : torch.Size([1, 64, 32, 32])
Depthwise+Pointwise çıktı şekli  : torch.Size([1, 64, 32, 32])


## 5. Pointwise Convolution'un Kullanım Alanları

**1. Kanal Genişletme / Daraltma**  
- `C_in → C_out` dönüşümünü en hafif şekilde yapar.  
- Bottleneck yapılarda (ResNet, MobileNetV2, MBConv) kritik roldedir.

**2. Kanal Karıştırma (Channel Mixing)**  
- Depthwise sonrası kanalların birbirinden bilgi almasını sağlar.  
- Temsil gücünü ciddi şekilde artırır.

**3. Özellik Birleştirme (Feature Fusion)**  
- Farklı "branch"lerden gelen feature map'leri birleştirirken,  
  kanallar üzerinden esnek dönüşümler tanımlamayı sağlar.

**4. Modern Blok Tasarımları**  
- MBConv (MobileNetV2 / EfficientNet)  
- SE-Block öncesi / sonrası kanal dönüşümleri  
- Ghost Module (GhostNet)  
- ShuffleNet gibi hafif yapılarda sık kullanılır.


## 6. Kısa Özet

- **Pointwise Convolution = 1×1 Convolution**.  
- Uzamsal boyutları **değiştirmez**; sadece kanallar üzerinde çalışır.  
- Ana görevleri:
  - Kanal sayısını değiştirmek (C_in → C_out)
  - Kanalları birbirine karıştırmak (channel mixing)
- Depthwise Conv ile birlikte kullanıldığında:
  - Uzamsal işlem (depthwise)
  - Kanal karışımı (pointwise)
  çok daha düşük maliyetle elde edilir.
- PyTorch'ta pointwise yazmak için kalıp:

  ```python
  nn.Conv2d(C_in, C_out, kernel_size=1, stride=1, padding=0)
  ```

Bu yapı sayesinde, modern CNN mimarilerinde hem **verimli** hem de **ifadeli** feature dönüşümleri tasarlamak mümkün hale gelir.
